# Tweepy 前面一樣

In [1]:
import tweepy

consumer_key = 'LkyO4Vh1kCYXqirDoFU3JDvJA'
consumer_secret = 'tZpOair4VdijJUdUfoWwOU6UupAn9Zn7RdkaoOv6Qbr2ifC2x5'
access_token = '1250757816781946882-zrerLif1LrLB6oyiPVmXuNCIOlkRMV'
access_token_secret = 'wpNsHDk42L7hJSaLFbyAMqsNCKaV9T8SCeoDpdZQCFKDg'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

#獲取類似於內容控制代碼的東西
api = tweepy.API(auth)

import os
import pandas as pd
import tweepy
import re
import string
from textblob import TextBlob
import preprocessor as p
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

consumer_key = 'LkyO4Vh1kCYXqirDoFU3JDvJA'
consumer_secret = 'tZpOair4VdijJUdUfoWwOU6UupAn9Zn7RdkaoOv6Qbr2ifC2x5'
access_token = '1250757816781946882-zrerLif1LrLB6oyiPVmXuNCIOlkRMV'
access_token_secret = 'wpNsHDk42L7hJSaLFbyAMqsNCKaV9T8SCeoDpdZQCFKDg'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

#獲取類似於內容控制代碼的東西
api = tweepy.API(auth)

In [2]:
#columns of the csv file
COLS = ['id', 'created_at', 'source', 'original_text','clean_text', 'sentiment','polarity','subjectivity', 'lang',
        'favorite_count', 'retweet_count', 'original_author', 'possibly_sensitive', 'hashtags',
        'user_mentions', 'place', 'place_coord_boundaries']

#set two date variables for date range
start_date = '2020-01-01'
end_date = '2020-04-20'

# Happy Emoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])

# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])

#Emoji patterns
emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)

#combine sad and happy emoticons
emoticons = emoticons_happy.union(emoticons_sad)

In [3]:
def clean_tweets(tweet):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(tweet)

    #after tweepy preprocessing the colon left remain after removing mentions
    #or RT sign in the beginning of the tweet
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
    #replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)


    #remove emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)

    #filter using NLTK library append it to a string
    filtered_tweet = [w for w in word_tokens if not w in stop_words]
    filtered_tweet = []

    #looping through conditions
    for w in word_tokens:
        #check tokens against stop words , emoticons and punctuations
        if w not in stop_words and w not in emoticons and w not in string.punctuation:
            filtered_tweet.append(w)
    return ' '.join(filtered_tweet)

# 把cursor(api.search中的tweet_mode改為"extended)"爬全文

In [17]:
if os.path.exists("data/試試看2.csv"):
    df = pd.read_csv("data/試試看2.csv", header=0)
else:
    df = pd.DataFrame(columns=COLS)
    

for page in tweepy.Cursor(api.search, q="Dow Jones Industrial Average",
                              count=100, tweet_mode='extended', include_rts=False, since=start_date).pages(5):
        for status in page:
            new_entry = []
            status = status._json
            ## check whether the tweet is in english or skip to the next tweet
            if status['lang'] != 'en':
                continue
                
            
            # 如果是replying to就會直接跳出
            if status['full_text'][0] == "R" and status['full_text'][1] == "T":
                continue
            #when run the code, below code replaces the retweet amount and
            #no of favorires that are changed since last download.
            if status['created_at'] in df['created_at'].values:
                i = df.loc[df['created_at'] == status['created_at']].index[0]
                if status['favorite_count'] != df.at[i, 'favorite_count'] or \
                   status['retweet_count'] != df.at[i, 'retweet_count']:
                    df.at[i, 'favorite_count'] = status['favorite_count']
                    df.at[i, 'retweet_count'] = status['retweet_count']
                continue
            filtered_tweet=clean_tweets(status['full_text'])
            blob = TextBlob(filtered_tweet)
            Sentiment = blob.sentiment

            #seperate polarity and subjectivity in to two variables
            polarity = Sentiment.polarity
            subjectivity = Sentiment.subjectivity

            #new entry append
            new_entry += [status['id'], status['created_at'],
                          status['source'], status['full_text'],filtered_tweet, Sentiment,polarity,subjectivity, status['lang'],
                          status['favorite_count'], status['retweet_count']]

            #to append original author of the tweet
            new_entry.append(status['user']['screen_name'])

            try:
                is_sensitive = status['possibly_sensitive']
            except KeyError:
                is_sensitive = None
            new_entry.append(is_sensitive)

            # hashtagas and mentiones are saved using comma separted
            hashtags = ", ".join([hashtag_item['text'] for hashtag_item in status['entities']['hashtags']])
            new_entry.append(hashtags)
            mentions = ", ".join([mention['screen_name'] for mention in status['entities']['user_mentions']])
            new_entry.append(mentions)

            #get location of the tweet if possible
            try:
                location = status['user']['location']
            except TypeError:
                location = ''
            new_entry.append(location)

            try:
                coordinates = [coord for loc in status['place']['bounding_box']['coordinates'] for coord in loc]
            except TypeError:
                coordinates = None
            new_entry.append(coordinates)

            single_tweet_df = pd.DataFrame([new_entry], columns=COLS)
            df = df.append(single_tweet_df, ignore_index=True)
            csvFile = open("data/試試看2.csv", 'a' ,encoding='utf-8')
        df.to_csv(csvFile, mode='a', columns=COLS, index=False, encoding="utf-8")





### 根據polarity正負(情緒為正面抑或是負面)分裝成兩個csv檔

##### 先造出一個正向情緒的csv檔案

In [25]:
if os.path.exists("data/positive_news.csv"):
    df = pd.read_csv("data/positive_news.csv", header=0)
else:
    df = pd.DataFrame(columns=COLS)

for page in tweepy.Cursor(api.search, q="Dow Jones Industrial Average",
                              count=200, tweet_mode='extended', include_rts=False, since=start_date).pages(10000):
    for status in page:
        new_entry = []
        status = status._json
        # 如果不是英文就直接跳過這則貼文
        if status['lang'] != 'en':
            continue
        # 如果是replying to某人貼文也先跳過
        if status['full_text'][0] == "R" and status['full_text'][1] == "T":
            continue

        
        # 這邊做df預處理
        if status['created_at'] in df['created_at'].values:
            i = df.loc[df['created_at'] == status['created_at']].index[0]
            if status['favorite_count'] != df.at[i, 'favorite_count'] or \
                status['retweet_count'] != df.at[i, 'retweet_count']:
                df.at[i, 'favorite_count'] = status['favorite_count']
                df.at[i, 'retweet_count'] = status['retweet_count']
            continue
        
        filtered_tweet=clean_tweets(status['full_text'])
        blob = TextBlob(filtered_tweet)
        Sentiment = blob.sentiment
        
        #seperate polarity and subjectivity in to two variables
        polarity = Sentiment.polarity
        subjectivity = Sentiment.subjectivity

        #new entry append
        new_entry += [status['id'], status['created_at'],
                        status['source'], status['full_text'],filtered_tweet, Sentiment,polarity,subjectivity, status['lang'],
                        status['favorite_count'], status['retweet_count']]

        #to append original author of the tweet
        new_entry.append(status['user']['screen_name'])

        try:
            is_sensitive = status['possibly_sensitive']
        except KeyError:
            is_sensitive = None
        new_entry.append(is_sensitive)

        # hashtagas and mentiones are saved using comma separted
        hashtags = ", ".join([hashtag_item['text'] for hashtag_item in status['entities']['hashtags']])
        new_entry.append(hashtags)
        mentions = ", ".join([mention['screen_name'] for mention in status['entities']['user_mentions']])
        new_entry.append(mentions)

        #get location of the tweet if possible
        try:
            location = status['user']['location']
        except TypeError:
            location = ''
        new_entry.append(location)

        try:
            coordinates = [coord for loc in status['place']['bounding_box']['coordinates'] for coord in loc]
        except TypeError:
            coordinates = None
        new_entry.append(coordinates)
        if polarity >= 0:
            single_tweet_df = pd.DataFrame([new_entry], columns=COLS)
            df = df.append(single_tweet_df, ignore_index=True)
        csvFile = open("data/positive_news.csv", 'a' ,encoding='utf-8')
    df.to_csv(csvFile, mode='a', columns=COLS, index=False, encoding="utf-8")

       

##### 負面情緒

In [4]:
if os.path.exists("data/negative_news.csv"):
    df = pd.read_csv("data/negative_news.csv", header=0)
else:
    df = pd.DataFrame(columns=COLS)

for page in tweepy.Cursor(api.search, q="Dow Jones Industrial Average",
                              count=200, tweet_mode='extended', include_rts=False, since=start_date).pages(10000):
    for status in page:
        new_entry = []
        status = status._json
        # 如果不是英文就直接跳過這則貼文
        if status['lang'] != 'en':
            continue
        # 如果是replying to某人貼文也先跳過
        if status['full_text'][0] == "R" and status['full_text'][1] == "T":
            continue

        
        # 這邊做df預處理
        if status['created_at'] in df['created_at'].values:
            i = df.loc[df['created_at'] == status['created_at']].index[0]
            if status['favorite_count'] != df.at[i, 'favorite_count'] or \
                status['retweet_count'] != df.at[i, 'retweet_count']:
                df.at[i, 'favorite_count'] = status['favorite_count']
                df.at[i, 'retweet_count'] = status['retweet_count']
            continue
        
        filtered_tweet=clean_tweets(status['full_text'])
        blob = TextBlob(filtered_tweet)
        Sentiment = blob.sentiment
        
        #seperate polarity and subjectivity in to two variables
        polarity = Sentiment.polarity
        subjectivity = Sentiment.subjectivity

        #new entry append
        new_entry += [status['id'], status['created_at'],
                        status['source'], status['full_text'],filtered_tweet, Sentiment,polarity,subjectivity, status['lang'],
                        status['favorite_count'], status['retweet_count']]

        #to append original author of the tweet
        new_entry.append(status['user']['screen_name'])

        try:
            is_sensitive = status['possibly_sensitive']
        except KeyError:
            is_sensitive = None
        new_entry.append(is_sensitive)

        # hashtagas and mentiones are saved using comma separted
        hashtags = ", ".join([hashtag_item['text'] for hashtag_item in status['entities']['hashtags']])
        new_entry.append(hashtags)
        mentions = ", ".join([mention['screen_name'] for mention in status['entities']['user_mentions']])
        new_entry.append(mentions)

        #get location of the tweet if possible
        try:
            location = status['user']['location']
        except TypeError:
            location = ''
        new_entry.append(location)

        try:
            coordinates = [coord for loc in status['place']['bounding_box']['coordinates'] for coord in loc]
        except TypeError:
            coordinates = None
        new_entry.append(coordinates)
        if polarity < 0:
            single_tweet_df = pd.DataFrame([new_entry], columns=COLS)
            df = df.append(single_tweet_df, ignore_index=True)
        csvFile = open("data/negative_news.csv", 'a' ,encoding='utf-8')
    df.to_csv(csvFile, mode='a', columns=COLS, index=False, encoding="utf-8")

       